# Big Data for Engineers – Moodle

# Fall 2022 -  Week 11 - Rumble



In [1]:
import requests
import json
import time
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def rumble(line, cell=None):
    if cell is None:
        data = line
    else:
        data = cell

    start = time.time()                                                         
    response = json.loads(requests.post(server, data=data).text)                   
    end = time.time()                                                              
    print("Took: %s s" % (end - start))

    if 'warning' in response:
        print(json.dumps(response['warning']))
    if 'values' in response:
        for e in response['values']:
            print(json.dumps(e))
    elif 'error-message' in response:
        return response['error-message']
    else:
        return response

In [2]:
server = 'http://localhost:8001/jsoniq'

In [9]:
%%rumble
count(for $i in json-file("/home/confusion-100000.json")
return $i)

Took: 23.599609851837158 s
100000


# Moodle Graded Exercise 11.1

And now to the actual Moodle queries. Here we still use the language game dataset (please use the **small** dataset for these questions: confusion-100000.json).


1\. Find the number of games where the guessed language is wrong (meaning not equal to the target one) and that language is German.

In [16]:
%%rumble
count(for $i in json-file("/home/confusion-100000.json", 10)
where $i.target ne $i.guess and $i.guess eq "German"
return $i)

Took: 1.3057677745819092 s
128


In [17]:
%%rumble
count(for $i in json-file("/home/confusion-100000.json", 10)
where $i.target ne $i.guess and $i.target eq "German"
return $i)

Took: 1.4782190322875977 s
132


2\. Return the number of distinct "target" languages.

In [25]:
%%rumble
count(distinct-values(json-file("/home/confusion-100000.json", 10).target))

Took: 1.1889379024505615 s
68


# Moodle Graded Exercise 11.2

3\. Return the sample IDs (i.e., the sample field) of the top two (2) games where the guessed language is wrong (equal to not the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In [31]:
%%rumble
for $i in json-file("/home/confusion-100000.json", 10)
where $i.guess ne $i.target
order by $i.guess, $i.country, $i.date
count $c
where $c <= 2
return $i.sample

Took: 2.0265440940856934 s
"bd3eefed0e12797c6e2b45b8db8d88d9"
"5e609abac81edbf76a58bc1d2a850f12"


In [29]:
%%rumble
for $i in json-file("/home/confusion-100000.json", 10)
where $i.guess ne $i.target
order by $i.target, $i.country, $i.date
count $c
where $c <= 2
return $i.sample

Took: 2.118534803390503 s
"00b85faa8b878a14f8781be334deb137"
"efcd813daec1c836d9f030b30caa07ce"


4\. Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three least frequent country/language combinations.

In [ ]:
%%rumble
for $i in json-file("/home/confusion-100000.json", 10)
group by $country := $i.country, $target := $i.target
order by count($i)
return { "country": $country, "target": $target, "count": count($i) }

In [44]:
%%rumble
for $i in json-file("/home/confusion-100000.json", 10)
count $c
where $c <= 2
return $i.choices

Took: 0.41781115531921387 s
["Maori", "Mandarin", "Norwegian", "Tongan"]
["Danish", "Dinka", "Khmer", "Lao"]


# Moodle Graded Exercise 11.3

5\. Find the percentage of games where (the answer was correct && the correct guess was the tenth choice amongst the array of possible answers).

In [55]:
%%rumble
let $count := count(
    for $i in json-file("/home/confusion-100000.json", 10)
    where $i.target eq $i.guess and $i.target eq $i.choices[][10]
    return $i
)
let $full := count(
    for $i in json-file("/home/confusion-100000.json", 10)
    return $i
)
return ($count div $full)

Took: 1.9885931015014648 s
8e-05


# Moodle Graded Exercise 11.4

6\. Sort the languages by decreasing overall percentage of wrong guesses and return the first three languages.

In [61]:
%%rumble
let $filtered := (
    for $i in json-file("/home/confusion-100000.json", 10)
    where $i.target ne $i.guess
    group by $language := $i.target 
    return {"language": $language, "count_filtered": count($i)}
)
let $full := (
    for $i in json-file("/home/confusion-100000.json", 10)
    group by $language := $i.target 
    return {"language": $language, "count_full": count($i)}
)
let $join := (
    for $a in $filtered, $b in $full
    where $a."language" = $b."language"
    return {
        "language" : $a."language",
        "fraction" : $a."count_filtered" div $b."count_full"
    }
)
for $j in $join
order by $j.fraction descending
count $c   
where $c <= 3
return $j

Took: 4.042201995849609 s
{"language": "Dinka", "fraction": 0.5805946792}
{"language": "Fijian", "fraction": 0.5736325386}
{"language": "Kannada", "fraction": 0.5653450808}
